# 🧠 RADE: Retriever-Augmented Document Entity Extraction

**Note: FAISS GPU install is needed for this notebook.**

This notebook demonstrates how to extract information from trust documents using the **RADE** pipeline.  
Each query retrieves relevant chunks using **ColBERT**, then:
- Runs a **QA model** (RoBERTa) to extract precise answers
- Uses **GLiNER** for entity extraction tied to each query

***
## Section 1:

This section demonstrates how to use the **RADE** (Retriever-Augmented Document Entity Extraction) system for parsing and indexing an unstructured pdf documents

RADE integrates:
- **ColBERT** for semantic retrieval
- **Azure Document Intelligence** for document parsing (PDFs & scanned pages)

---

## 🗂️ Steps:
1. **Initialize RADE** with model names and Azure credentials
2. **Add a document** (PDF) — scanned or digital
3. **Build the ColBERT index**
4. **Retrieve top-k relevant pages**


In [11]:
#required imports
import sys
from rade import RADE, RetrievedPage, DocumentPage
from dotenv import load_dotenv
import json
import os

### 🔧 Step 1 – Initialize RADE

In [2]:
#obtain azure doc intel keys and endpoint
load_dotenv() #must be called to load Azure doc intel keys
key = os.environ.get("DOC_INTEL_Key")
endpoint = os.environ.get("DOC_INTEL_ENDPOINT")
if not key or not key.strip():
    raise ValueError("❌ Initialization failed: 'key' must not be empty.")
if not endpoint or not endpoint.strip():
    raise ValueError("❌ Initialization failed: 'endpoint' must not be empty.")

In [3]:
#initialize
rade = RADE(doc_int_key=key, doc_int_endpoint=endpoint)

Using device map: {'retrieval': 'cuda', 'qa': 'cuda'}
Initializing retrieval model...


/opt/conda/lib/python3.11/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


Initializing QA model and entity extraction models...


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Models initialized successfully!


## 📄 Step 2 – Add a PDF Document

In [4]:
# ✅ Define additional QA Queries
pdf_path = "data/living-trust-forms-03_Ileana Hardison Rev Family Trust DTD 05052010.pdf"
rade.add_document(pdf_path, 1)

📄 Loading document: 1


📚 Step 3 – Build the Index

In [5]:
import os
os.environ.get("DOC_INTEL", "").strip()

''

In [6]:
rade.build_faiss_index()

🔧 Building index for 17 pages...
---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[May 14, 15:32:33] #> Note: Output directory .ragatouille/colbert/indexes/living-trust-forms-03_Ileana Hardison Rev Family Trust DTD 05052010.pdf already exists


#> Starting...
#> Starting...


/opt/conda/lib/python3.11/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/opt/conda/lib/python3.11/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/opt/conda/lib/python3.11/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/opt/conda/lib/python3.11/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextMana

nranks = 2 	 num_gpus = 2 	 device=1
[May 14, 15:32:38] [1] 		 #> Encoding 18 passages..
nranks = 2 	 num_gpus = 2 	 device=0
[May 14, 15:32:38] [0] 		 #> Encoding 20 passages..
[May 14, 15:32:40] [0] 		 avg_doclen_est = 123.32221984863281 	 len(local_sample) = 20
[May 14, 15:32:40] [1] 		 avg_doclen_est = 123.32221984863281 	 len(local_sample) = 18
[May 14, 15:32:40] [0] 		 Creating 1,024 partitions.
[May 14, 15:32:40] [0] 		 *Estimated* 4,686 embeddings.
[May 14, 15:32:40] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/living-trust-forms-03_Ileana Hardison Rev Family Trust DTD 05052010.pdf/plan.json ..


/opt/conda/lib/python3.11/site-packages/colbert/indexing/collection_indexer.py:256: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sub_sample = torch.load(sub_sample_path)
WA

Clustering 4459 points in 128D to 1024 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
[May 14, 15:32:41] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


...

[May 14, 15:32:42] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/opt/conda/lib/python3.11/site-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/colbert/indexing/codecs/residual.py:141: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_s

[May 14, 15:32:43] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[0.027, 0.027, 0.026, 0.023, 0.022, 0.027, 0.025, 0.022, 0.025, 0.027, 0.023, 0.029, 0.025, 0.025, 0.026, 0.027, 0.024, 0.023, 0.025, 0.026, 0.026, 0.025, 0.024, 0.026, 0.023, 0.025, 0.025, 0.027, 0.022, 0.024, 0.025, 0.026, 0.029, 0.025, 0.026, 0.023, 0.027, 0.027, 0.026, 0.032, 0.027, 0.023, 0.023, 0.025, 0.027, 0.026, 0.024, 0.027, 0.027, 0.025, 0.025, 0.026, 0.025, 0.024, 0.022, 0.027, 0.029, 0.026, 0.031, 0.026, 0.023, 0.024, 0.027, 0.024, 0.023, 0.026, 0.024, 0.024, 0.024, 0.026, 0.027, 0.025, 0.027, 0.021, 0.027, 0.028, 0.027, 0.026, 0.025, 0.026, 0.027, 0.028, 0.026, 0.027, 0.024, 0.027, 0.025, 0.024, 0.025, 0.03, 0.026, 0.025, 0.028, 0.028, 0.028, 0.026, 0.031, 0.026, 0.027, 0.025, 0.027, 0.03, 0.026, 0.023, 0.029, 0.027, 0.023, 0.025, 0.024, 0.024, 0.023, 0.032, 0.026, 0.025, 0.03, 0.024, 0.027, 0.027, 0.024, 0.023, 0.024, 0.026, 0.026, 0.028, 0.028, 

100%|██████████| 1024/1024 [00:00<00:00, 36744.30it/s]
[rank0]:[W514 15:32:44.862566312 ProcessGroupNCCL.cpp:1250] Warning: WARNING: process group has NOT been destroyed before we destruct ProcessGroupNCCL. On normal program exit, the application should call destroy_process_group to ensure that any pending NCCL operations have finished in this process. In rare cases this process can exit before this point and block the progress of another member of the process group. This constraint has always been present,  but this warning has only been added since PyTorch 2.4 (function operator())


#> Joined...
#> Joined...
Done indexing!
✅ Index built: .ragatouille/colbert/indexes/living-trust-forms-03_Ileana Hardison Rev Family Trust DTD 05052010.pdf


🔍 Step 4 – Retrieve Top-k Pages for a Query

In [9]:
results = rade.retrieve_faiss_index(["What is the name of the Trustee of this trust?"], 3)
for r in results:
    print(f"📄 Page {r.page.page_num} | Score: {r.score:.2f}")
    print(r.page.text[:400])


📄 Page 1 | Score: 22.97
With this purpose, the primary asset management goal for this Living Trust will be the protection of the value of the Property. The secondary asset management goal for this Living Trust is to generate income and growth at a reasonable risk.
Trustee
2. During the lifetime of the Grantor, and while the Grantor is not Incapacitated, the Grantor will serve as the primary trustee (the "Primary Trustee"
📄 Page 9 | Score: 22.34
b. After the death of the Grantor, the Trustee will have the power to appoint one or more individuals or institutions to act as co-Trustee where it is deemed reasonable and in the best overall interest of this Living Trust.
c. The Trustee may employ and rely on the advice of experts including, but not limited to, legal counsel, accountants and investment advisors to help in the management of the P
📄 Page 10 | Score: 22.23
NONPUBLIC//FDIC INTERNAL ONLY
d. The Trustee may retain, exchange, insure, repair, improve, sell or dispose of any and all pe

---

## Section 2 🔍 RADE Trust Document Analyzer -  Entity and QA Extraction from Trust Documents

This section supports:
- Named Entity Recognition (NER) using GLiNER for labeled entity queries
- Question Answering (QA) using RoBERTa for direct questions

Each result:
- Retrieves top-k chunks via ColBERT
- Runs either QA or NER based on the query type
- Presents results interactively using arrows to flip through top passages



### Step1 🧩 – Define Query-to-Label Mapping

Use the schema as below:

```python
{
  "Who are the Grantors?": {
    "type": "ner",
    "labels": ["Grantor_Settlor"]
  },
  "Who are the primary Trustees?": {
    "type": "ner",
    "labels": ["Primary_Trustee"]
  },
  "Who are the Successor Trustees?": {
    "type": "ner",
    "labels": ["Successor_Secondary_Trustee"]
  },
  "Who are the primary Beneficiaries?": {
    "type": "ner",
    "labels": ["Primary Beneficiary"]
  },
  "Who are the Successor Beneficiaries?": {
    "type": "ner",
    "labels": ["Successor_Beneficiary"]
  },
  "What is the name of the trust?": {
    "type": "qa",
    "labels": []
  },
  "What is the date of the trust?": {
    "type": "qa",
    "labels": []
  },
  "Is this trust revocable or irrevocable?": {
    "type": "qa",
    "labels": []
  }
}



In [36]:
#read query file
query_file = "data/query_plan.json"
with open(query_file, "r") as j:
    query_plan = json.load(j)


In [29]:
#function to use RADE extractive model
def extract_entities_per_page(rade, retrieved_pages, labels):
    page_entities = {}

    for rp in retrieved_pages:
        text = rp.page.text.replace("\n", " ")
        entities = rade.entity_extraction_model.predict_entities(text, labels)
        key = f"{rp.page.doc_name}_page{rp.page.page_num}"
        page_entities[key] = entities

    return page_entities


## 🧠 Step 2 – Run RADE: Parse + Index + Query

In [61]:
query_results = []

for query, meta in query_plan.items():
    retrieved = rade.retrieve_faiss_index([query], k=3)  # List of RetrievedPage
    documents = [r.page.text for r in retrieved]

    # Rerank using ColBERT
    retrieved_reranked = rade.retrieval_model.rerank(query=[query], documents=documents, k=3)

    # Map reranked results back to original RetrievedPage objects using result_index
    reranked_results = [retrieved[item['result_index']] for item in retrieved_reranked]

    if meta["type"] == "qa":
        qa_result = rade.run_qa_pipeline(
            question=query,
            retrieved_texts=[
                {
                    "content": rp.page.text,
                    "document_metadata": {
                        "document": rp.page.doc_name,
                        "page": rp.page.page_num
                    }
                } for rp in reranked_results
            ]
        )
        query_results.append({
            "query": query,
            "type": "qa",
            "labels": [],
            "results": reranked_results,
            "qa": qa_result,
            "entities": {}
        })

    elif meta["type"] == "ner":
        retrieved_texts = [
            {
                "content": rp.page.text,
                "document_metadata": {
                    "document": rp.page.doc_name,
                    "page": rp.page.page_num
                }
            } for rp in reranked_results
        ]

        entity_result = rade.extract_entities_with_gliner(
            retrieved_texts=retrieved_texts,
            labels=meta["labels"],
            threshold=0.3
        )

        query_results.append({
            "query": query,
            "type": "ner",
            "labels": meta["labels"],
            "results": reranked_results,
            "qa": None,
            "entities": {
                f"{r['document']}_page{r['page']}": [
                    e for e in entity_result["entities"]
                    if e in entity_result["entities"]
                ]
                for r in entity_result["retrieved"]
            }
        })


100%|██████████| 1/1 [00:00<00:00, 69.59it/s]


In [62]:
def normalize_retrieved_pages(data):
    normalized = []
    for item in data:
        normalized.append({
            "query": item['query'],
            "type": item['type'],
            "labels": item['labels'],
            "results": [
                {
                    "doc_id": r.page.doc_id,
                    "doc_name": r.page.doc_name,
                    "page_num": r.page.page_num,
                    "text": r.page.text
                } for r in item['results']
            ]
        })
    return normalized


In [70]:
normalized_data = normalize_retrieved_pages(query_results)

with open("./data/rade_query_result.json", "w") as f:
    json.dump(normalized_data, f, indent=4)

## Step 3 - Visualize

In [71]:
import html
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, clear_output

def highlight_entities(text: str, entities: list, highlight_answer: str = None) -> str:
    """
    Highlight QA answer and NER entities using bright yellow <mark>.
    """
    text = html.escape(text)

    # Highlight QA answer (first)
    if highlight_answer:
        safe_answer = html.escape(highlight_answer.strip())
        if safe_answer and safe_answer in text:
            text = text.replace(
                safe_answer,
                f'<mark style="background-color:yellow" title="QA Answer">{safe_answer}</mark>'
            )

    # Highlight NER entities
    entities = sorted(entities, key=lambda e: -len(e['text']))
    for ent in entities:
        safe_text = html.escape(ent["text"])
        label = html.escape(ent["label"])
        score = ent.get("score", None)
        tooltip = f"{label}" + (f" ({score:.2f})" if score else "")
        text = text.replace(
            safe_text,
            f'<mark style="background-color:yellow" title="{tooltip}">{safe_text}</mark>'
        )

    return text


def show_query_results(query_result: dict):
    query = query_result["query"]
    query_type = query_result["type"]
    pages = query_result["results"]
    entity_map = query_result["entities"]
    qa = query_result["qa"]

    index = widgets.IntText(value=0, layout=widgets.Layout(width="40px"), disabled=True)
    output = widgets.Output()

    prev_button = widgets.Button(description="◀️ Prev", layout=widgets.Layout(width="80px"))
    next_button = widgets.Button(description="Next ▶️", layout=widgets.Layout(width="80px"))
    nav_box = widgets.HBox([prev_button, index, next_button])

    def render_page(i):
        with output:
            clear_output(wait=True)
            page = pages[i].page
            score = pages[i].score
            key = f"{page.doc_name}_page{page.page_num}"
            ents = entity_map.get(key, []) if query_type == "ner" else []

            display(Markdown(f"## 🔎 Query: `{query}`"))

            if query_type == "qa":
                answer = qa.get("answer", "[No answer]")
                display(Markdown(f"### 🤖 QA Answer: `{answer}`"))
            else:
                answer = None  # Not used

            display(Markdown(f"**📄 Page {page.page_num} — Score: `{score:.2f}` — File: `{page.doc_name}`**"))

            highlighted_text = highlight_entities(
                text=page.text[:3000],
                entities=ents,
                highlight_answer=answer
            )

            display(HTML(f"<pre style='line-height:1.5;font-family:monospace'>{highlighted_text}</pre>"))

    def on_prev_clicked(_):
        if index.value > 0:
            index.value -= 1
            render_page(index.value)

    def on_next_clicked(_):
        if index.value < len(pages) - 1:
            index.value += 1
            render_page(index.value)

    prev_button.on_click(on_prev_clicked)
    next_button.on_click(on_next_clicked)

    display(nav_box, output)
    render_page(index.value)


In [72]:
for result in query_results:
    show_query_results(result)



Output()

Output()

Output()

Output()

Output()

Output()

Output()

Output()